<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/vector_stores/postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postgres Vector Store
In this notebook we are going to show how to use [Postgresql](https://www.postgresql.org) and  [pgvector](https://github.com/pgvector/pgvector)  to perform vector searches in LlamaIndex

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
# import logging
# import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
import textwrap

### Setup OpenAI
The first step is to configure the openai key. It will be used to run inference.

Once we switched to local model we don't need this.

In [2]:
import os
import dotenv

# import openai

# # Reload the variables in your '.env' file (override the existing variables)
# dotenv.load_dotenv("../.env", override=True)
# openai.api_key = os.environ["OPENAI_API_KEY"] 

Local Embedding Models

The easiest way to use a local model is:

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

### Loading documents
Load the documents stored in the `data/faculty_websites/` using the SimpleDirectoryReader.

Change the documents passed based on your need!

You can refer to this to see how to load files in different ways e.g. entire directory

https://docs.llamaindex.ai/en/stable/examples/data_connectors/simple_directory_reader.html

In [5]:
from llama_index.core import SimpleDirectoryReader
# reader = SimpleDirectoryReader(
#     input_files=["./data/faculty_websites/Bhiksharaj_lti_page.txt"]
# )
reader = SimpleDirectoryReader(input_dir="./data/faculty_websites/")
docs = reader.load_data()
# print(f"Loaded {len(docs)} docs")
print("Document ID:", docs[0].doc_id)

Document ID: c1ff526d-1c9c-4fe2-81c7-254b9776a07a


### Create the Database
Using an existing postgres running at localhost, create the database we'll be using.

In [6]:
import psycopg2

# Reload the variables in your '.env' file (override the existing variables)
dotenv.load_dotenv("../.env", override=True)
pwd = os.environ['PG_PASSWORD_RAG']
user = "711-rag"
connection_string = f'dbname=postgres user={user} password={pwd}'
db_name = "711-rag"
conn = psycopg2.connect(connection_string)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS \"{db_name}\"")
    c.execute(f"CREATE DATABASE \"{db_name}\"")

### Create the index
Here we create an index backed by Postgres using the documents loaded previously. PGVectorStore takes a few arguments.

In [7]:
from sqlalchemy import make_url

# url = make_url(connection_string)
vector_store = PGVectorStore.from_params(
    database=db_name,
    host="localhost",
    password=pwd,
    port=5432,
    user=user,
    table_name="all",
    embed_dim=384, 
)

You might experience error because pgvector extension is not enabled for the database. You can go to terminal and run

```
sudo -u postgres psql
\c "711-rag" # This connects to the 711-rag database we just created
CREATE EXTENSION vector;
\q
```

In [8]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
# ollama
Settings.llm = Ollama(model="llama2", request_timeout=30.0)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context, show_progress=True
)

query_engine = index.as_query_engine()

Parsing nodes:   0%|          | 0/67 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/752 [00:00<?, ?it/s]

ValueError: A string literal cannot contain NUL (0x00) characters.

### Query the index
We can now ask questions using our index.

In [60]:
response = query_engine.query("What's the email of Bhiksha Raj?")

In [61]:
print(textwrap.fill(str(response), 100))

The email address of Bhiksha Raj is bhiksha@cs.cmu.edu.


### Querying existing index  (I have not tested the code from here and below!)

In [ ]:
vector_store = PGVectorStore.from_params(
    database="vector_db",
    host="localhost",
    password="password",
    port=5432,
    user="postgres",
    table_name="paul_graham_essay",
    embed_dim=1536,  # openai embedding dimension
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What did the author do?")

In [ ]:
print(textwrap.fill(str(response), 100))

The author worked on writing and programming before college. They wrote short stories and tried
writing programs on an IBM 1401 computer. They also built a microcomputer and started programming on
it, writing simple games and a word processor. In college, the author initially planned to study
philosophy but switched to AI due to their interest in intelligent computers. They taught themselves
AI by learning Lisp.


### Hybrid Search

To enable hybrid search, you need to:
1. pass in `hybrid_search=True` when constructing the `PGVectorStore` (and optionally configure `text_search_config` with the desired language)
2. pass in `vector_store_query_mode="hybrid"` when constructing the query engine (this config is passed to the retriever under the hood). You can also optionally set the `sparse_top_k` to configure how many results we should obtain from sparse text search (default is using the same value as `similarity_top_k`).

In [ ]:
from sqlalchemy import make_url

url = make_url(connection_string)
hybrid_vector_store = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="paul_graham_essay_hybrid_search",
    embed_dim=1536,  # openai embedding dimension
    hybrid_search=True,
    text_search_config="english",
)

storage_context = StorageContext.from_defaults(
    vector_store=hybrid_vector_store
)
hybrid_index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

/Users/suo/dev/llama_index/llama_index/vector_stores/postgres.py:217: SAWarning: TypeDecorator TSVector() will not produce a cache key because the ``cache_ok`` attribute is not set to True.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this type object's state is safe to use in a cache key, or False to disable this warning. (Background on this warning at: https://sqlalche.me/e/20/cprf)
  session.commit()


In [ ]:
hybrid_query_engine = hybrid_index.as_query_engine(
    vector_store_query_mode="hybrid", sparse_top_k=2
)
hybrid_response = hybrid_query_engine.query(
    "Who does Paul Graham think of with the word schtick"
)

In [ ]:
print(hybrid_response)

Roy Lichtenstein


### PgVector Query Options

#### IVFFlat Probes

Specify the number of [IVFFlat probes](https://github.com/pgvector/pgvector?tab=readme-ov-file#query-options) (1 by default)

When retrieving from the index, you can specify an appropriate number of IVFFlat probes (higher is better for recall, lower is better for speed)

In [ ]:
retriever = index.as_retriever(
    vector_store_query_mode=query_mode,
    similarity_top_k=top_k,
    vector_store_kwargs={"ivfflat_probes": 10},
)

#### HNSW EF Search

Specify the size of the dynamic [candidate list](https://github.com/pgvector/pgvector?tab=readme-ov-file#query-options-1) for search (40 by default)

In [ ]:
retriever = index.as_retriever(
    vector_store_query_mode=query_mode,
    similarity_top_k=top_k,
    vector_store_kwargs={"hnsw_ef_search": 300},
)